In [ ]:
# https://stackoverflow.com/questions/47805170/whats-the-hardware-spec-for-google-colaboratory

In [ ]:
!df -h

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!cat /proc/meminfo

# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [ ]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'

Спарсим текст из файлика.\
**`Requests`** [tutorial](https://realpython.com/python-requests/)

In [ ]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

In [ ]:
resp

## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [ ]:
text = text.lower()

Разобьём весь текст на предложения. \
**`re`** [tutorial](https://tproger.ru/translations/regular-expression-python/)

In [ ]:
import re 
# выкидываем лишние символы! 
text = re.sub('\n|\t|\r', ' ', text)

**`nltk`** [tutorial](https://www.guru99.com/nltk-tutorial.html)

In [ ]:
import nltk

# nltk.download('all') # если хотим всё и сразу 

# нам хватит вот этого: 
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)

len(sents)

In [ ]:
sents[220]

Разобьём каждое предложение на отдельные слова.

In [ ]:
from nltk.tokenize import word_tokenize

print(word_tokenize(sents[0]), '\n')
sents[0]

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

In [ ]:
# разбейте все предложения на токены 
sents_tokenize  =  [tokenizer.tokenize(item) for item in sents]
sents_tokenize[:2]

In [ ]:
# Flatten без numpy :) 
words = [item for sent in sents_tokenize for item in sent]
words[:10]

# Конструкция выше аналогично этому
# words = []
# for sent in sents_tokenize:
#     for item in sent:
#         if True:
#             words.append(item)

In [ ]:
len(words) # всего слов

In [ ]:
len(set(words)) # уникальных слов

Можно выбросить все стоп-слова. 

In [ ]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

In [ ]:
len(stopwords_ru)

In [ ]:
# избавьтесь от стоп-слов 
sents_tokenize = [[item for item in sent if item not in stopwords_ru]
                        for sent in sents_tokenize]
sents_tokenize[:2]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [ ]:
!pip install pymorphy2

In [ ]:
import pymorphy2


morph = pymorphy2.MorphAnalyzer()

text = "Филипп пошёл в Авиньон и пленил пап!"
tokens = tokenizer.tokenize(text)  # regexp tokenizer initialized earlier

" ".join(morph.normal_forms(token)[0] for token in tokens)

In [ ]:
p = morph.parse('стали')
p

Зачем нужно возвращать словарь в такой конструкции: `morph.normal_forms(token)[0]`? Ответ ниже.

In [ ]:
morph.normal_forms('стали')

Обработаем все слова из датасета. 

In [ ]:
# лемматизируйте все слова из датасета
sents_tokenize = [[morph.normal_forms(item)[0] for item in sent] 
                    for sent in sents_tokenize]

In [ ]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [ ]:
len(words) # всего слов

In [ ]:
len(set(words)) # уникальных слов

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочется больше предобработки, хороший мануал по [ссылке](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb).  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [ ]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

In [ ]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `vector_size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `epochs` — количество эпох обучения,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.


**`gensim`** [documentation](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py)\
[Нововведения](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4) в **gensim 4.0**


In [ ]:
!pip install gensim==4.1.2

In [ ]:
type(sents_tokenize), len(sents_tokenize)
# sents_tokenize[0][:5]

In [ ]:
#%%time 
from gensim.models.word2vec import Word2Vec

# vector_size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(vector_size=100, window=2, min_count=3, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели (еще так можно модель дообучать)
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

Другой способ инициализировать и тренировать модель.

Такой синтаксис под капотом точно так же строит словарь и вызывает метод `train`

In [ ]:
model2 = Word2Vec(sentences=sents_tokenize, vector_size=100, window=2, min_count=3, workers=-1, epochs=100)
print(model2)

In [ ]:
model.corpus_count # число примеров в обучающей выборке

Смотрим, сколько в модели слов.

In [ ]:
len(model.wv), len(model2.wv)

In [ ]:
# model.wv.key_to_index

In [ ]:
'старуха' in model.wv.key_to_index, 'старуха' in model2.wv.key_to_index

In [ ]:
model.wv.get_vecattr("старуха", "count"), model2.wv.get_vecattr("старуха", "count") 

## 3. Свойства модели

In [ ]:
# вектор слова
model.wv['старуха'][:10], model2.wv['старуха'][:10]

In [ ]:
len(model.wv['старуха'])

In [ ]:
# размерность вектора
model.wv['старуха'].shape, model2.wv['старуха'].shape

In [ ]:
# похожести слов 
model.wv.similarity('тварь', 'право'), model2.wv.similarity('тварь', 'право')

In [ ]:
# самые похожие
model.wv.most_similar('топор', topn=10)

In [ ]:
# арифметика
model.wv.most_similar(positive=['раскольников','соня'], 
                       negative=['тварь'])[:10]

### KeyedVectors

**TL;DR:** the main difference is that KeyedVectors do not support further training. On the other hand, by shedding the internal data structures necessary for training, KeyedVectors offer a smaller RAM footprint and a simpler interface.

[Source](https://radimrehurek.com/gensim/models/keyedvectors.html#)


По сути, KeyedVectors нужны для экономии места на диске при сохранении модели

In [ ]:
# model.save('aaaaa')

# tmp = Word2Vec.load('aaaaa')

In [ ]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

vector = wv['раскольников']  # Get numpy vector of a word
vector.shape, vector

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [ ]:
model_path = "./our_w2v.model"
model.save(model_path)

In [ ]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [ ]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

Предобработка.

In [ ]:
# А теперь ваш код предобработки

sents2 = sent_tokenize(text2.lower())
sents_tokenize2  =  [tokenizer.tokenize(item) for item in sents2]
sents_tokenize2 = [[item for item in sent if item not in stopwords_ru]
                        for sent in sents_tokenize2]
                        
# лемматизируйте все слова из датасета
sents_tokenize2 = [[morph.normal_forms(item)[0] for item in sent] 
                    for sent in sents_tokenize2]

In [ ]:
sents_tokenize2[10]

In [ ]:
len(sents_tokenize2)

Дополняем модель.

In [ ]:
# И теперь ваш код обучения
model.train(corpus_iterable=sents_tokenize2, total_examples=len(sents_tokenize2), epochs=model.epochs)

In [ ]:
model.corpus_count # число примеров в обучающей выборке

In [ ]:
'ядро' in model.wv

In [ ]:
'ядро' in our_model.wv

In [ ]:
# our_model.wv.most_similar('ядро')

Пример со старым словом.

In [ ]:
our_model.wv.most_similar('сын')

In [ ]:
model.wv.most_similar('сын')